In [1]:
import argparse
import json
import multiprocessing as mp
import os
import pickle
import platform
import queue
import random
import sys
import warnings
import time
import copy
from pathlib import Path
from collections import defaultdict
from typing import List, Sequence, Set, Dict, Optional, Any, cast

sys.path.insert(0, os.getcwd())

import compress_pickle
import numpy as np
import torch
import tqdm
from ai2thor.controller import Controller
from ai2thor.platform import CloudRendering

from allenact.utils.misc_utils import md5_hash_str_as_int
from allenact_plugins.ithor_plugin.ithor_environment import IThorEnvironment
from datagen.datagen_utils import (
    get_random_seeds,
    find_object_by_type,
    scene_from_type_idx,
    remove_objects_until_all_have_identical_meshes,
    check_object_opens,
    filter_pickupable,
    mapping_counts,
    open_objs
)
from datagen.datagen_constants import NUM_TEST_TASKS, NUM_TRAIN_TASKS, NUM_VAL_TASKS, PICKUPABLE_RECEPTACLE_PAIRS, PICKUP_OBJECTS_FOR_TEST, GOOD_4_ROOM_HOUSES, STAGE_TO_DEST_NUM_SCENES, STAGE_TO_VALID_TASK_TO_SCENES, STAGE_TO_VALID_TASKS
from env.constants import IOU_THRESHOLD, OBJECT_TYPES_WITH_PROPERTIES, SCENE_TYPE_TO_SCENES, STARTER_HOME_SERVICE_DATA_DIR, THOR_COMMIT_ID
from env.environment import (
    HomeServiceEnvironment,
    HomeServiceTaskSpec,
)
from env.utils import extract_obj_data


args = argparse.Namespace()
args.debug = True
args.mode = "train"



/home/yhkim/miniconda3/envs/robot-home-service/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# task_type: None
stage_seeds = get_random_seeds()

stage_to_tasks = {
    stage: STAGE_TO_VALID_TASKS[stage]
    for stage in [args.mode]
}
house_count = 1
scene_count = 1

partition = "train" if args.mode == "train" else "valid"
idxs = [0,]
stage_to_tasks = {
    "debug": [
        stage_to_tasks[partition][idx]
        for idx in idxs
    ]
}

stage_to_task_to_task_specs = {
    stage: {} for stage in stage_to_tasks
}

for stage in stage_to_tasks:
    for task in stage_to_tasks[stage]:
        if task not in stage_to_task_to_task_specs[stage]:
            stage_to_task_to_task_specs[stage][task] = [
                [-1] * scene_count
            ] * house_count
            

In [3]:
# Runner to work - task_type = find_limits
task_type = "find_limits"
task_info = dict(
    scene="",
    stage=stage,
    task=task,
    seed=stage_seeds[stage],
    house_count=house_count,
)

(
    scene,
    seed,
    stage,
    task,
    house_count,
    # scene_i,
    # obj_name_to_avoid_positions,
) = (
    task_info["scene"],
    task_info["seed"],
    task_info["stage"],
    task_info["task"],
    task_info["house_count"],
    # task_info["scene_i"],
    # task_info["obj_name_to_avoid_positions"],
)

mode = task.split("_")[0]


# Worker init env
env = HomeServiceEnvironment(
    force_cache_reset=True,
    controller_kwargs={
        "scene": "Procedural",
        "x_display": None,
        "platform": CloudRendering,
    },
)
env._houses.mode(mode)

from datagen.datagen_runner import find_scene_to_limits_for_task

data = find_scene_to_limits_for_task(
    stage_seed=seed,
    scene=scene,
    env=env,
    stage=stage,
    task=task,
    house_count=house_count,
)

Loading test: 100%|██████████| 1000/1000 [00:00<00:00, 8883.00it/s]


Task train train_pick_and_place_AlarmClock_Dresser for train_7772 limits


In [4]:
# Worker to Runner: task_type = "find_limits"
resuls = data
scene = list(data.keys())[0]
limits = list(data.values())[0]
task_info["scene"] = scene
task_info["limits"] = limits
task_info["house_i"] = 0
task_info["scene_i"] = 0
task_info["house_count"] = 1
task_info["scene_count"] = 1
task_type = "home_service"

In [6]:
# Runner to work - task_type = home_service
(
    scene,
    seed,
    stage,
    task,
    house_i,
    scene_i,
    limits,
    house_count,
    scene_count,
    # obj_name_to_avoid_positions,
) = (
    task_info["scene"],
    task_info["seed"],
    task_info["stage"],
    task_info["task"],
    task_info["house_i"],
    task_info["scene_i"],
    task_info["limits"],
    task_info["house_count"],
    task_info["scene_count"],
    # task_info["obj_name_to_avoid_positions"],
)

mode, idx = tuple(scene.split("_"))
env._houses.mode(mode)

In [7]:
# generate_home_service_episode_for_task in datagen.datagen_runner
stage_seed=seed
force_visible: bool = True
place_stationary: bool = False
rotation_increment: int = 90
allow_moveable_in_goal_randomization: bool = False
object_types_to_not_move: Set[str] = set()

if 360 % rotation_increment != 0:
    raise ValueError("Rotation increment must be a factor of 360")

controller = env.controller
pick, recep = task.split("_")[-2], task.split("_")[-1]

print(f"Task {stage} {task} for {scene}")

seed = md5_hash_str_as_int(f"{stage_seed}|{task}|{scene}")
random.seed(seed)

Task debug train_pick_and_place_AlarmClock_Dresser for train_7772


In [8]:
if not env.procthor_reset(scene_name=scene, force_reset=True):
    print(f"Cannot reset scene {scene}")


In [9]:
if not remove_objects_until_all_have_identical_meshes(controller):
    print(
        f"Failed to remove_objects_until_all_have_identical_meshes in {scene}"
    )

In [10]:
scene_has_openable = 0 != len(
    [
        o
        for o in controller.last_event.metadata["objects"]
        if o["openable"] and not o["pickupable"]
    ]
)

In [11]:
scene_has_openable

True

In [12]:
evt = controller.step("GetReachablePositions")
rps: List[Dict[str, float]] = evt.metadata["actionReturn"][:]
rps.sort(key=lambda d: (round(d["x"], 2), round(d["z"], 2)))
rotations = np.arange(0, 360, rotation_increment)


In [13]:
room_to_rps = copy.deepcopy(env.room_to_reachable_positions())
for room, room_rps in room_to_rps.items():
    room_rps.sort(key=lambda d: (round(d["x"], 2), round(d["z"], 2)))

In [15]:
all_objects = env.ids_to_objs()
possible_starting_rooms = sorted(list(env.room_to_poly.keys()))
possible_target_rooms = []
if recep != "User":
    possible_target_rooms = [
        room_id
        for room_id, rids in limits["room_receptacles"].items()
        for rid in rids
        if all_objects[rid]["objectType"] == recep
    ]
st = random.getstate()
random.seed(1234567)
random.shuffle(possible_starting_rooms)
random.shuffle(possible_target_rooms)
random.setstate(st)

In [16]:
possible_starting_rooms

['room|5', 'room|6', 'room|4', 'room|7']

In [17]:
possible_target_rooms

['room|4']

In [18]:
limits["room_receptacles"]["room|4"]

['Bed_19_0',
 'Chair_215_1_0',
 'Chair_215_1_1',
 'Chair_215_1_2',
 'Desk_311_1_0',
 'Dining_Table_201_1_0',
 'RoboTHOR_dresser_birkeland_0']

In [19]:
starting_room = cast(str, possible_starting_rooms[scene_i % len(possible_starting_rooms)])
if recep != "User":
    target_room = cast(str, possible_target_rooms[scene_i % len(possible_target_rooms)])
else:
    target_room = starting_room

In [22]:
pos = random.choice(room_to_rps[starting_room])
rot = {"x": 0, "y": int(random.choice(rotations)), "z": 0}
init_kwargs = {
    "randomSeed": random.randint(0, int(1e7) - 1),
    "forceVisible": force_visible,
    "placeStationary": place_stationary,
    "excludedReceptacles": ["ToiletPaperHanger"],
    "allowMoveable": allow_moveable_in_goal_randomization,
}


In [23]:
# generate_one_home_service_given_initial_conditions in datagen.datagen_runner
from datagen.datagen_runner import generate_one_home_service_given_initial_conditions
(
    starting_poses,
    target_poses,
    objs_to_open,
) = generate_one_home_service_given_initial_conditions(
    env=env,
    scene=scene,
    task=task,
    init_kwargs=init_kwargs,
    agent_pos=pos,
    agent_rot=rot,
    starting_room=starting_room,
    target_room=target_room,
    # single_room=single_room,
    # object_types_to_not_move=object_types_to_not_move,
    # allow_putting_objects_away=MAX_TRIES >= try_count >= MAX_TRIES // 2,
    # possible_openable_ids=limits["room_openables"][single_room]
    # if single_room in limits["room_openables"]
    # else [],
)

In [27]:
task_spec_dict = {
    "pickup_object": pick,
    "target_receptacle": recep,
    "agent_position": pos,
    "agent_rotation": int(rot["y"]),
    "starting_poses": starting_poses,
    "target_poses": target_poses,
    "objs_to_open": objs_to_open,
}

In [28]:
# HomeServiceEnvironment.reset in env.environment
# HomeServiceEnvironment._task_spec_reset in env.environment

task_spec = HomeServiceTaskSpec(
    scene=scene,
    **task_spec_dict,
)
force_reset: bool = True
close_doors: bool = False
merge_rooms: bool = True
house: Optional[Dict[str, Any]] = None

env.procthor_reset(
    task_spec.scene,
    force_reset=force_reset,
    close_doors=close_doors,
    merge_rooms=merge_rooms,
    house=house,
)

True

In [30]:
if env._enhanced_physics_determinism:
    env.controller.step("PausePhysicsAutoSim")

In [31]:
env.controller.last_event

<ai2thor.server.Event at 0x7f8942ab2430
    .metadata["lastAction"] = PausePhysicsAutoSim
    .metadata["lastActionSuccess"] = True
    .metadata["errorMessage"] = "
    .metadata["actionReturn"] = None
>

In [32]:
obj_ids = [
    obj["objectId"]
    for obj in env.objects()
    if obj["objectType"]
    not in ["Wall", "Floor", "Window", "Doorway", "Doorframe", "Room"]
]

In [34]:
env.controller.step(
    action="SetObjectFilter", objectIds=obj_ids,
)

<ai2thor.server.Event at 0x7f89584ea790
    .metadata["lastAction"] = SetObjectFilter
    .metadata["lastActionSuccess"] = True
    .metadata["errorMessage"] = "
    .metadata["actionReturn"] = None
>

In [35]:
len(env.controller.last_event.metadata["objects"])

99

In [36]:
pos = task_spec.agent_position
rot = {"x": 0, "y": task_spec.agent_rotation, "z": 0}
env.controller.step(
    "TeleportFull",
    **pos,
    rotation=rot,
    horizon=0.0,
    standing=True,
    forceAction=True,
)

<ai2thor.server.Event at 0x7f888ad8f130
    .metadata["lastAction"] = TeleportFull
    .metadata["lastActionSuccess"] = True
    .metadata["errorMessage"] = "
    .metadata["actionReturn"] = None
>

In [39]:
[o['name'] for o in task_spec.starting_poses] == [o['name'] for o in task_spec.target_poses]

True

In [44]:
names = [o['name'] for o in task_spec.starting_poses]
onames = [o['name'] for o in env.objects()]

enames = [n for n in onames if n not in names]

In [46]:
enames

['Wall_Decor_Photo_2_0',
 'Wall_Decor_Photo_1_0',
 'Wall_Decor_Photo_1V_0',
 'Wall_Decor_Painting_3V_0',
 'Wall_Decor_Photo_6_0',
 'Wall_Decor_Photo_3V_0',
 'Wall_Decor_Photo_9_0',
 'Wall_Decor_Painting_8_0',
 'bin_23_0',
 'Houseplant_18_0',
 'Shelf_Open (1)',
 'Shelf_Open',
 'Dining_Table_23_2_0',
 'Sofa_220_1_0',
 'Television_13_0',
 'TV_Stand_222_1_0',
 'Houseplant_18_1',
 'bin_17_0',
 'Chair_311_1_0',
 'Chair_311_1_1',
 'Chair_311_1_2',
 'Chair_311_1_3',
 'Vase_Medium_4_0',
 'Dining_Table_218_1_0',
 'Fridge_22_0',
 'Microwave_5_0',
 'Countertop_C_8x10_0',
 'bin_20_0',
 'Bathroom_Faucet_30_0',
 'SinkBasin',
 'Sink_19_0',
 'Toilet_1_0',
 'Chair_215_1_0',
 'Chair_215_1_1',
 'Chair_215_1_2',
 'Desk_Lamp_4_0',
 'Houseplant_10_0',
 'Vase_Open_1_0',
 'Dining_Table_201_1_0',
 'Desk_Lamp_2_0',
 'Desk_311_1_0',
 'Drawer (5)',
 'Drawer (4)',
 'Drawer (3)',
 'Drawer (2)',
 'Drawer (1)',
 'Drawer',
 'RoboTHOR_dresser_birkeland_0',
 'Bed_19_0']

In [47]:
env.controller.step(
    "SetObjectPoses",
    objectPoses=task_spec.starting_poses,
    **env.set_object_poses_params(),
)

<ai2thor.server.Event at 0x7f888a883370
    .metadata["lastAction"] = SetObjectPoses
    .metadata["lastActionSuccess"] = True
    .metadata["errorMessage"] = "
    .metadata["actionReturn"] = 
>

In [48]:
task_spec.starting_poses[0]

{'name': 'Alarm_Clock_21_0',
 'objectName': 'Alarm_Clock_21_0',
 'position': {'x': 0.44407251477241516,
  'y': 0.8022906184196472,
  'z': 7.238447189331055},
 'rotation': {'x': 359.7735290527344,
  'y': 0.0005084187141619623,
  'z': 0.0017071663169190288}}

In [49]:
ppp = {}
for it, (sp, tp) in enumerate(zip(task_spec.starting_poses, task_spec.target_poses)):
    position_dist = IThorEnvironment.position_dist(
        sp["position"], tp["position"]
    )
    rotation_dist = IThorEnvironment.angle_between_rotations(
        sp["rotation"], tp["rotation"]
    )
    same = False
    if position_dist < 1e-2 and rotation_dist < 10.0:
        same = True

    ppp[it] = {
        'p': position_dist,
        'r': rotation_dist,
        's': same,
    }

In [50]:
[it for it, p in ppp.items() if not p['s']]

[0]

In [51]:
task_spec.target_poses[0]

{'name': 'Alarm_Clock_21_0',
 'objectName': 'Alarm_Clock_21_0',
 'position': {'x': 0.31954631209373474,
  'y': 1.1283433437347412,
  'z': 4.968244552612305},
 'rotation': {'x': 359.7821044921875,
  'y': 0.003348764032125473,
  'z': 0.0017180170398205519}}

In [53]:
task_spec.starting_poses[0]

{'name': 'Alarm_Clock_21_0',
 'objectName': 'Alarm_Clock_21_0',
 'position': {'x': 0.44407251477241516,
  'y': 0.8022906184196472,
  'z': 7.238447189331055},
 'rotation': {'x': 359.7735290527344,
  'y': 0.0005084187141619623,
  'z': 0.0017071663169190288}}

In [55]:
env.pose_inconsistency(pose_spec_list=task_spec.target_poses)

In [60]:
from env.utils import get_pose_info

o = env.ids_to_objs()["Alarm_Clock_21_0"]
env_pose = cast(Dict[str, Any], get_pose_info(o))


In [64]:
shuffled_name_to_spec = {o["name"]: o for o in task_spec.target_poses}
spec_pose = dict(env_pose)
spec_pose["rotation"] = shuffled_name_to_spec["Alarm_Clock_21_0"]["rotation"]
spec_pose["position"] = shuffled_name_to_spec["Alarm_Clock_21_0"]["position"]

In [65]:
env_pose

{'type': 'AlarmClock',
 'position': {'x': 0.44407251477241516,
  'y': 0.8022906184196472,
  'z': 7.238447189331055},
 'rotation': {'x': 359.7735290527344,
  'y': 0.0005084187723696232,
  'z': 0.0017071664333343506},
 'openness': None,
 'pickupable': True,
 'broken': False,
 'objectId': 'Alarm_Clock_21_0',
 'name': 'Alarm_Clock_21_0',
 'parentReceptacles': ['Desk_311_1_0'],
 'bounding_box': [[0.5317944288253784, 0.8015761375427246, 7.256286144256592],
  [0.35635095834732056, 0.8015708923339844, 7.256287574768066],
  [0.35635054111480713, 0.8013831377029419, 7.208789348602295],
  [0.531794011592865, 0.8013883829116821, 7.20878791809082],
  [0.5317895412445068, 0.9651243686676025, 7.255639553070068],
  [0.356346070766449, 0.9651191234588623, 7.255640983581543],
  [0.35634565353393555, 0.9649313688278198, 7.2081427574157715],
  [0.5317891240119934, 0.9649366140365601, 7.208141326904297]]}

In [70]:
[o for o in task_spec.starting_poses if env.ids_to_objs()[o['name']]["openable"]]

[{'name': 'Book_21_0',
  'objectName': 'Book_21_0',
  'position': {'x': 7.6402177810668945,
   'y': 0.7813732028007507,
   'z': 5.335953235626221},
  'rotation': {'x': 0.019056078046560287,
   'y': 180.0003204345703,
   'z': 0.2329062819480896}},
 {'name': 'Box_19_0',
  'objectName': 'Box_19_0',
  'position': {'x': 2.910229206085205,
   'y': 0.9422257542610168,
   'z': 12.04285717010498},
  'rotation': {'x': 359.9901428222656,
   'y': 270.0,
   'z': -0.0008050284232012928}},
 {'name': 'Laptop_18_0',
  'objectName': 'Laptop_18_0',
  'position': {'x': 4.631888389587402,
   'y': 0.6782166361808777,
   'z': 8.956149101257324},
  'rotation': {'x': -3.648087555782262e-14,
   'y': 1.0253411346639041e-05,
   'z': 4.077082280673494e-07}},
 {'name': 'Laptop_7_0',
  'objectName': 'Laptop_7_0',
  'position': {'x': 3.103160858154297,
   'y': 0.7236921787261963,
   'z': 12.30789566040039},
  'rotation': {'x': 1.6806190615170635e-05,
   'y': 180.00003051757812,
   'z': -8.065858128247783e-05}},
 {'na

In [67]:
env.are_poses_equal(spec_pose, env_pose, min_iou=0.9, open_tol=0.05, treat_broken_as_unequal=True)

True

In [71]:
a = [{'1': 123, '2': {'x': 1, 'y': 3, 'z': 5}, '3': {'x': 2, 'y': 4, 'z': 6},}]
b = [{'1': 123, '2': {'x': 3, 'y': 2, 'z': 1}, '3': {'x': 7, 'y': 6, 'z': 5},}]

for o0, o1 in zip(a, b):
    o0['2'] = o1['2']



In [72]:
a

[{'1': 123, '2': {'x': 3, 'y': 2, 'z': 1}, '3': {'x': 2, 'y': 4, 'z': 6}}]

In [73]:
b

[{'1': 123, '2': {'x': 3, 'y': 2, 'z': 1}, '3': {'x': 7, 'y': 6, 'z': 5}}]